In [1]:
import sqlalchemy
import pandas as pd
import server.model.connection as smc
import server.view.dataframes as smd

In [2]:
smd.ranking_df(12).head()

phase       auto                                                            \
actor      robot                                                             
task  placeScale                                                             
         matches sum_successes max_successes min_successes count_successes   
team                                                                         
108         10.0           1.0           1.0           0.0             3.0   
1165        10.0           1.0           1.0           0.0             5.0   
1287        10.0           2.0           1.0           1.0             2.0   
1296        10.0           0.0           0.0           0.0             1.0   
1318         NaN           NaN           NaN           NaN             NaN   

phase                                                                      \
actor                                                                       
task                                                                        
      avg_successes sum_attempts max_attempts min_attempts count_attempts   
team                                                                        
108        0.333333          3.0          1.0          1.0            3.0   
1165       0.200000          5.0          1.0          1.0            5.0   
1287       1.000000          2.0          1.0          1.0            2.0   
1296       0.000000          1.0          1.0          1.0            1.0   
1318            NaN          NaN          NaN          NaN            NaN   

phase        ...               teleop                                         \
actor        ...                robot                                          
task         ...          placeSwitch                                          
             ...        avg_successes sum_attempts max_attempts min_attempts   
team         ...                                                               
108          ...                  5.6         75.0         14.0          1.0   
1165         ...                  5.7         70.0         13.0          1.0   
1287         ...                  3.9         52.0          9.0          2.0   
1296         ...                  6.9         85.0         18.0          3.0   
1318         ...                  5.6         75.0         14.0          3.0   

phase                                                                   \
actor                                                                    
task                                                                     
      count_attempts avg_attempts max_cycle_times min_cycle_times  avg   
team                                                                     
108             10.0          7.5             0.0             0.0  0.0   
1165            10.0          7.0             0.0             0.0  0.0   
1287            10.0          5.2             0.0             0.0  0.0   
1296            10.0          8.5             0.0             0.0  0.0   
1318            10.0          7.5             0.0             0.0  0.0   

phase                    
actor                    
task                     
      count_cycle_times  
team                     
108                10.0  
1165               10.0  
1287               10.0  
1296               10.0  
1318               10.0  

[5 rows x 75 columns]

In [2]:
conn = smc.engine.connect()

In [4]:
sql = sqlalchemy.text(
    "with current AS (SELECT s.event_id as event_id, s.match, date from schedules sched, "
    "status s WHERE sched.event_id = s.event_id "
    "AND sched.match = s.match limit 1 ) "
    "SELECT * FROM current;")
pd.read_sql(sql, conn)

,event_id,match,date
0,25127,112-q,2017-04-21T18:20:00


In [5]:
num_matches = 12
sql = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc) "
    
    "SELECT * from recent_matches;"
)
pd.read_sql(sql, conn).head(20)

,r,id,date,level,match,alliance,team,station,event_id
0,1,51953,2017-04-21T18:20:00,qual,112-q,red,108,1,25127
1,2,51834,2017-04-21T15:40:00,qual,092-q,red,108,2,25127
2,3,51758,2017-04-21T13:56:00,qual,079-q,blue,108,1,25127
3,4,51729,2017-04-21T13:16:00,qual,074-q,blue,108,2,25127
4,5,51625,2017-04-21T09:04:00,qual,057-q,red,108,3,25127
5,6,51592,2017-04-21T08:16:00,qual,051-q,blue,108,3,25127
6,7,51500,2017-04-20T15:12:00,qual,036-q,blue,108,1,25127
7,8,51432,2017-04-20T11:42:00,qual,025-q,red,108,2,25127
8,9,51389,2017-04-20T10:46:00,qual,018-q,red,108,1,25127
9,10,51309,2017-04-20T08:54:00,qual,004-q,blue,108,2,25127


In [6]:
select_sum = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc), "
    
    "team_match_count as ( "
    "select team, count(team) as team_matches from recent_matches group by team"
       ") "
       
    "SELECT * FROM team_match_count;"
)
pd.read_sql(select_sum, conn).head()

,team,team_matches
0,2990,10
1,6225,10
2,3794,10
3,3019,10
4,6554,10


In [7]:
evt_id = 25127
sql = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc), "
    
    "team_match_count as ( "
    "select team, count(team) as team_matches from recent_matches group by team"
       ") "
    
    "SELECT teams.name AS team, phases.name AS phase, tasks.name AS task, actors.name AS actor, "
    "MAX(team_match_count.team_matches) AS matches, "
    "SUM(successes) AS sum_successes, MAX(successes) as max_successes, MIN(successes) as min_successes, "
    "COUNT(successes) AS count_successes, AVG(successes) as avg_successes, "
    "SUM(attempts) AS sum_attempts, MAX(attempts) as max_attempts, MIN(attempts) AS min_attempts, "
    "COUNT(attempts) AS count_attempts, AVG(attempts) as avg_attempts, "
    "MAX(cycle_times) AS max_cycle_times, MIN(cycle_times) AS min_cycle_times, "
    "AVG(cycle_times), COUNT(cycle_times) AS count_cycle_times "
    "FROM ((((teams FULL OUTER JOIN measures ON teams.id=measures.team_id) "
    "LEFT JOIN tasks ON tasks.id = measures.task_id) "
    "LEFT JOIN phases ON phases.id = measures.phase_id) "
    "LEFT JOIN events ON events.id = measures.event_id) "
    "LEFT JOIN actors ON actors.id = measures.actor_id "
    "LEFT JOIN matches ON matches.id = measures.match_id "
    "LEFT JOIN team_match_count ON team_match_count.team = teams.name "
    "RIGHT JOIN recent_matches ON recent_matches.match = matches.name AND team_match_count.team = teams.name "
    "AND recent_matches.team = team_match_count.team "
    "WHERE events.id = " + str(evt_id) + " AND actors.name<> 'alliance' "
    "GROUP BY teams.name, tasks.name, phases.name, actors.name "
    "ORDER BY teams.name, phases.name, tasks.name, actors.name;"
)
df = pd.read_sql(sql, conn)
df.head(10)

,team,phase,task,actor,matches,sum_successes,max_successes,min_successes,count_successes,avg_successes,sum_attempts,max_attempts,min_attempts,count_attempts,avg_attempts,max_cycle_times,min_cycle_times,avg,count_cycle_times
0,108,auto,placeScale,robot,10,1,1,0,3,0.333333,3,1,1,3,1.0,0,0,0.0,3
1,108,auto,placeSwitch,robot,10,1,1,0,4,0.250000,4,1,1,4,1.0,0,0,0.0,4
2,108,auto,startPosition,robot,10,0,0,0,10,0.000000,0,0,0,10,0.0,0,0,0.0,10
3,108,teleop,placeScale,robot,10,17,3,0,10,1.700000,50,9,3,10,5.0,0,0,0.0,10
4,108,teleop,placeSwitch,robot,10,56,12,1,10,5.600000,75,14,1,10,7.5,0,0,0.0,10
5,1165,auto,placeScale,robot,10,1,1,0,5,0.200000,5,1,1,5,1.0,0,0,0.0,5
6,1165,auto,placeSwitch,robot,10,2,1,1,2,1.000000,2,1,1,2,1.0,0,0,0.0,2
7,1165,auto,startPosition,robot,10,0,0,0,10,0.000000,0,0,0,10,0.0,0,0,0.0,10
8,1165,teleop,placeScale,robot,10,16,4,0,10,1.600000,43,8,1,10,4.3,0,0,0.0,10
9,1165,teleop,placeSwitch,robot,10,57,13,1,10,5.700000,70,13,1,10,7.0,0,0,0.0,10
